# WG: Process Precip for H0

Process synthetic precipitation time series for H0 analysis

In [1]:
%matplotlib inline

In [2]:
import os
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats as sstats
import sys
import datetime as dt

In [3]:
sys.path.append( "D:\Repositories\WeatherGenerator" )

In [4]:
import WG_Inputs as WGI

In [5]:
IN_DIR = r'C:\Temp\WG_Test_Out\Test2\Processed'
H0_ROOT = "H0PrecipAllReal_G"
H0P1_ROOT = "H0P1PrecipAllReal_G"
H0P2_ROOT = "H0P2PrecipAllReal_G"
H0P3_ROOT = "H0P3PrecipAllReal_G"
H1_ROOT = "H1PrecipAllReal_G"
PICKLE_EXT = "pickle"
OUT_ROOT = "DC_WGMN2"

## Load and Compare the Data Period

In [6]:
StartDT = dt.datetime( 1981, 1, 1)
gCnt = 0
for gG in WGI.LOCA_KEYS:
    InFileName = "%s%d.%s" % ( H0_ROOT, gG, PICKLE_EXT )
    InFile = os.path.normpath( os.path.join( IN_DIR, InFileName ) )
    H0DF = pd.read_pickle( InFile, compression='zip' )
    InFileName = "%s%d.%s" % ( H1_ROOT, gG, PICKLE_EXT )
    InFile = os.path.normpath( os.path.join( IN_DIR, InFileName ) )
    H1DF = pd.read_pickle( InFile, compression='zip' )
    CompData = H1DF.loc[StartDT:] - H0DF.loc[StartDT:]
    OutDF = CompData.describe()
    # now write our worksheet
    OutFilePath = os.path.normpath( os.path.join( IN_DIR, "%s_DPeriodComp_Precip.xlsx" % OUT_ROOT ) )
    if gCnt == 0:
        with pd.ExcelWriter( OutFilePath, engine="openpyxl", mode='w' ) as writer:
            OutDF.to_excel( writer, sheet_name="Comp_%d" % gG, na_rep=str(np.nan) )
    else:
        with pd.ExcelWriter( OutFilePath, engine="openpyxl", mode='a' ) as writer:
            OutDF.to_excel( writer, sheet_name="Comp_%d" % gG, na_rep=str(np.nan) )
    # increment our counter
    gCnt += 1

## H0 Data Period

In [7]:
InFileName = "%s%d.%s" % ( H0_ROOT, WGI.LOCA_KEYS[0], PICKLE_EXT )
InFile = os.path.normpath( os.path.join( IN_DIR, InFileName ) )
H0DF = pd.read_pickle( InFile, compression='zip' )

In [8]:
RealColumns = H0DF.columns.to_list()

First do all of our set up stuff

In [9]:
OurQs = np.array( [ round( 0.05 * x, 2 ) for x in range(21) ], dtype=np.float32 )
NumQs = len( OurQs )
NumQs

21

In [10]:
StatsHdsList = list()
for iI in range(NumQs):
    cQ = OurQs[iI]
    Hdr = "%d_ptile" % round(cQ * 100.0)
    StatsHdsList.append( Hdr )
# end of for
StatsHdsList.append( "Average" )
StatsHdsList.append( "Variance" )
StatsHdsList.append( "Skew" )
StatsHdsList.append( "Kurt" )

In [11]:
DayYrList = [x for x in range(1, 367, 1)]
DayYrIndex = np.array( DayYrList, dtype=np.int32 )

In [12]:
MonthLister = [ "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", 
                 "Sep", "Oct", "Nov", "Dec", "All" ]

In [13]:
gCnt = 0
for gG in WGI.LOCA_KEYS:
    InFileName = "%s%d.%s" % ( H0_ROOT, gG, PICKLE_EXT )
    InFile = os.path.normpath( os.path.join( IN_DIR, InFileName ) )
    H0DF = pd.read_pickle( InFile, compression='zip' )
    # collect the overall statistics
    H0StatsList = list()
    npFlat = H0DF.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    H0StatsList.extend( QArray.tolist() )
    H0StatsList.append( npFlat.mean() )
    H0StatsList.append( npFlat.var() )
    H0StatsList.append( sstats.skew( npFlat ) )
    H0StatsList.append( sstats.kurtosis( npFlat ) )
    # do probabilistic time history
    DataDict = dict()
    for iI in range(NumQs):
        cQ = OurQs[iI]
        Hdr = "%d_ptile" % round(cQ * 100.0)
        DataDict[Hdr] = np.array( H0DF.quantile(q=cQ, axis=1), dtype=np.float32 )
    # end of for
    QuantsDF = pd.DataFrame( index=H0DF.index, data=DataDict )
    QuantsDF["Average"] = H0DF.mean(axis=1)
    QuantsDF["Variance"] = H0DF.var(axis=1)
    QuantsDF["Skew"] = H0DF.skew(axis=1)
    QuantsDF["Kurt"] = H0DF.kurtosis(axis=1)
    # now do day of the year distributions
    H0DF["IDayYear"] = H0DF.index.dayofyear
    DYList = list()
    for dD in DayYrList:
        LocalList = list()
        cDyOYrDF = H0DF[H0DF["IDayYear"] == dD].copy()
        cDyOYrDF.drop(columns="IDayYear", inplace=True )
        npFlat = cDyOYrDF.to_numpy().flatten(order='F')
        QArray = np.quantile( npFlat, OurQs )
        LocalList.extend( QArray.tolist() )
        LocalList.append( npFlat.mean() )
        LocalList.append( npFlat.var() )
        LocalList.append( sstats.skew( npFlat ) )
        LocalList.append( sstats.kurtosis( npFlat ) )
        DYList.append( LocalList )
    # end of for
    DataDict = dict()
    iCnt = 0
    for tHdr in StatsHdsList:
        DataDict[tHdr] = np.array( [ x[iCnt] for x in DYList ], dtype=np.float32 )
        iCnt += 1
    # end of for
    DailyQStatsDF = pd.DataFrame( index=DayYrIndex, data=DataDict )
    H0DF.drop( columns="IDayYear", inplace=True )
    # make a copy of H0DF for further manipulation
    H0DF["Month"] = H0DF.index.month
    PTH0DF = H0DF.copy()
    PTH0DF["Year"] = PTH0DF.index.year
    PTH0DF["Average"] = PTH0DF[RealColumns].mean(axis=1)
    PTH0DF["Max"] = PTH0DF[RealColumns].max(axis=1)
    PTH0DF["Min"] = PTH0DF[RealColumns].min(axis=1)
    PTH0DF.drop(columns=RealColumns, inplace=True)
    H0grouped = PTH0DF.groupby( ['Year', 'Month'], as_index=True )
    H0TSOutMeanDF = H0grouped[["Average"]].aggregate( [np.mean] )
    H0TSOutMaxDF = H0grouped[["Max"]].aggregate( [np.max] )
    H0TSOutMinDF = H0grouped[["Min"]].aggregate( [np.min] )
    PTMonthly = H0TSOutMaxDF.merge( H0TSOutMeanDF, how='inner', left_index=True, right_index=True )
    PTMonthly = PTMonthly.merge( H0TSOutMinDF, how='inner', left_index=True, right_index=True )
    # split into months
    H0Jan = H0DF[H0DF["Month"] == 1].copy()
    H0Jan.drop(columns="Month", inplace=True)
    H0Feb = H0DF[H0DF["Month"] == 2].copy()
    H0Feb.drop(columns="Month", inplace=True)
    H0Mar = H0DF[H0DF["Month"] == 3].copy()
    H0Mar.drop(columns="Month", inplace=True)
    H0Apr = H0DF[H0DF["Month"] == 4].copy()
    H0Apr.drop(columns="Month", inplace=True)
    H0May = H0DF[H0DF["Month"] == 5].copy()
    H0May.drop(columns="Month", inplace=True)
    H0Jun = H0DF[H0DF["Month"] == 6].copy()
    H0Jun.drop(columns="Month", inplace=True)
    H0Jul = H0DF[H0DF["Month"] == 7].copy()
    H0Jul.drop(columns="Month", inplace=True)
    H0Aug = H0DF[H0DF["Month"] == 8].copy()
    H0Aug.drop(columns="Month", inplace=True)
    H0Sep = H0DF[H0DF["Month"] == 9].copy()
    H0Sep.drop(columns="Month", inplace=True)
    H0Oct = H0DF[H0DF["Month"] == 10].copy()
    H0Oct.drop(columns="Month", inplace=True)
    H0Nov = H0DF[H0DF["Month"] == 11].copy()
    H0Nov.drop(columns="Month", inplace=True)
    H0Dec = H0DF[H0DF["Month"] == 12].copy()
    H0Dec.drop(columns="Month", inplace=True)
    MonAggList = list()
    # Jan
    LocalList = list()
    npFlat = H0Jan.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Feb
    LocalList = list()
    npFlat = H0Feb.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Mar
    LocalList = list()
    npFlat = H0Mar.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Apr
    LocalList = list()
    npFlat = H0Apr.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # May
    LocalList = list()
    npFlat = H0May.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # June
    LocalList = list()
    npFlat = H0Jun.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Jul
    LocalList = list()
    npFlat = H0Jul.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Aug
    LocalList = list()
    npFlat = H0Aug.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Sep
    LocalList = list()
    npFlat = H0Sep.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Oct
    LocalList = list()
    npFlat = H0Oct.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Nov
    LocalList = list()
    npFlat = H0Nov.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Dec
    LocalList = list()
    npFlat = H0Dec.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    MonAggList.append( H0StatsList )
    DataDict = dict()
    iCnt = 0
    for tHdr in StatsHdsList:
        DataDict[tHdr] = np.array( [ x[iCnt] for x in MonAggList ], dtype=np.float32 )
        iCnt += 1
    # end of for
    SummQStatsDF = pd.DataFrame( index=MonthLister, data=DataDict )
    # now write our worksheets
    OutFilePath = os.path.normpath( os.path.join( IN_DIR, "%s_H0Data_Precip.xlsx" % OUT_ROOT ) )
    if gCnt == 0:
        with pd.ExcelWriter( OutFilePath, engine="openpyxl" ) as writer:
            SummQStatsDF.to_excel( writer, sheet_name="Summary_%d" % gG, na_rep=str(np.nan), 
                                   columns=StatsHdsList )
            QuantsDF.to_excel( writer, sheet_name="Daily_Stats_%d" % gG, na_rep=str(np.nan), 
                               columns=StatsHdsList )
            DailyQStatsDF.to_excel( writer, sheet_name="DayYear_Stats_%d" % gG, na_rep=str(np.nan), 
                                    columns=StatsHdsList )
            PTMonthly.to_excel( writer, sheet_name="Monthly_Stats_%d" % gG, na_rep=str(np.nan) )
    else:
        with pd.ExcelWriter( OutFilePath, engine="openpyxl", mode='a' ) as writer:
            SummQStatsDF.to_excel( writer, sheet_name="Summary_%d" % gG, na_rep=str(np.nan), 
                                   columns=StatsHdsList )
            QuantsDF.to_excel( writer, sheet_name="Daily_Stats_%d" % gG, na_rep=str(np.nan), 
                               columns=StatsHdsList )
            DailyQStatsDF.to_excel( writer, sheet_name="DayYear_Stats_%d" % gG, na_rep=str(np.nan), 
                                    columns=StatsHdsList )
            PTMonthly.to_excel( writer, sheet_name="Monthly_Stats_%d" % gG, na_rep=str(np.nan) )
    # increment our counter
    gCnt += 1

### H0 CProj Period 1

Expect all of the H0 projection cases to be similar. We do have WGI.NUM_LOCA_GRID different locations. To do these, we need to loop through all grid locations for each one.

In [14]:
gCnt = 0
for gG in WGI.LOCA_KEYS:
    InFileName = "%s%d.%s" % ( H0P1_ROOT, gG, PICKLE_EXT )
    InFile = os.path.normpath( os.path.join( IN_DIR, InFileName ) )
    H0DF = pd.read_pickle( InFile, compression='zip' )
    # collect the overall statistics
    H0StatsList = list()
    npFlat = H0DF.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    H0StatsList.extend( QArray.tolist() )
    H0StatsList.append( npFlat.mean() )
    H0StatsList.append( npFlat.var() )
    H0StatsList.append( sstats.skew( npFlat ) )
    H0StatsList.append( sstats.kurtosis( npFlat ) )
    # do probabilistic time history
    DataDict = dict()
    for iI in range(NumQs):
        cQ = OurQs[iI]
        Hdr = "%d_ptile" % round(cQ * 100.0)
        DataDict[Hdr] = np.array( H0DF.quantile(q=cQ, axis=1), dtype=np.float32 )
    # end of for
    QuantsDF = pd.DataFrame( index=H0DF.index, data=DataDict )
    QuantsDF["Average"] = H0DF.mean(axis=1)
    QuantsDF["Variance"] = H0DF.var(axis=1)
    QuantsDF["Skew"] = H0DF.skew(axis=1)
    QuantsDF["Kurt"] = H0DF.kurtosis(axis=1)
    # now do day of the year distributions
    H0DF["IDayYear"] = H0DF.index.dayofyear
    DYList = list()
    for dD in DayYrList:
        LocalList = list()
        cDyOYrDF = H0DF[H0DF["IDayYear"] == dD].copy()
        cDyOYrDF.drop(columns="IDayYear", inplace=True )
        npFlat = cDyOYrDF.to_numpy().flatten(order='F')
        QArray = np.quantile( npFlat, OurQs )
        LocalList.extend( QArray.tolist() )
        LocalList.append( npFlat.mean() )
        LocalList.append( npFlat.var() )
        LocalList.append( sstats.skew( npFlat ) )
        LocalList.append( sstats.kurtosis( npFlat ) )
        DYList.append( LocalList )
    # end of for
    DataDict = dict()
    iCnt = 0
    for tHdr in StatsHdsList:
        DataDict[tHdr] = np.array( [ x[iCnt] for x in DYList ], dtype=np.float32 )
        iCnt += 1
    # end of for
    DailyQStatsDF = pd.DataFrame( index=DayYrIndex, data=DataDict )
    H0DF.drop( columns="IDayYear", inplace=True )
    # make a copy of H0DF for further manipulation
    H0DF["Month"] = H0DF.index.month
    PTH0DF = H0DF.copy()
    PTH0DF["Year"] = PTH0DF.index.year
    PTH0DF["Average"] = PTH0DF[RealColumns].mean(axis=1)
    PTH0DF["Max"] = PTH0DF[RealColumns].max(axis=1)
    PTH0DF["Min"] = PTH0DF[RealColumns].min(axis=1)
    PTH0DF.drop(columns=RealColumns, inplace=True)
    H0grouped = PTH0DF.groupby( ['Year', 'Month'], as_index=True )
    H0TSOutMeanDF = H0grouped[["Average"]].aggregate( [np.mean] )
    H0TSOutMaxDF = H0grouped[["Max"]].aggregate( [np.max] )
    H0TSOutMinDF = H0grouped[["Min"]].aggregate( [np.min] )
    PTMonthly = H0TSOutMaxDF.merge( H0TSOutMeanDF, how='inner', left_index=True, right_index=True )
    PTMonthly = PTMonthly.merge( H0TSOutMinDF, how='inner', left_index=True, right_index=True )
    # split into months
    H0Jan = H0DF[H0DF["Month"] == 1].copy()
    H0Jan.drop(columns="Month", inplace=True)
    H0Feb = H0DF[H0DF["Month"] == 2].copy()
    H0Feb.drop(columns="Month", inplace=True)
    H0Mar = H0DF[H0DF["Month"] == 3].copy()
    H0Mar.drop(columns="Month", inplace=True)
    H0Apr = H0DF[H0DF["Month"] == 4].copy()
    H0Apr.drop(columns="Month", inplace=True)
    H0May = H0DF[H0DF["Month"] == 5].copy()
    H0May.drop(columns="Month", inplace=True)
    H0Jun = H0DF[H0DF["Month"] == 6].copy()
    H0Jun.drop(columns="Month", inplace=True)
    H0Jul = H0DF[H0DF["Month"] == 7].copy()
    H0Jul.drop(columns="Month", inplace=True)
    H0Aug = H0DF[H0DF["Month"] == 8].copy()
    H0Aug.drop(columns="Month", inplace=True)
    H0Sep = H0DF[H0DF["Month"] == 9].copy()
    H0Sep.drop(columns="Month", inplace=True)
    H0Oct = H0DF[H0DF["Month"] == 10].copy()
    H0Oct.drop(columns="Month", inplace=True)
    H0Nov = H0DF[H0DF["Month"] == 11].copy()
    H0Nov.drop(columns="Month", inplace=True)
    H0Dec = H0DF[H0DF["Month"] == 12].copy()
    H0Dec.drop(columns="Month", inplace=True)
    MonAggList = list()
    # Jan
    LocalList = list()
    npFlat = H0Jan.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Feb
    LocalList = list()
    npFlat = H0Feb.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Mar
    LocalList = list()
    npFlat = H0Mar.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Apr
    LocalList = list()
    npFlat = H0Apr.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # May
    LocalList = list()
    npFlat = H0May.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # June
    LocalList = list()
    npFlat = H0Jun.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Jul
    LocalList = list()
    npFlat = H0Jul.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Aug
    LocalList = list()
    npFlat = H0Aug.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Sep
    LocalList = list()
    npFlat = H0Sep.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Oct
    LocalList = list()
    npFlat = H0Oct.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Nov
    LocalList = list()
    npFlat = H0Nov.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Dec
    LocalList = list()
    npFlat = H0Dec.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    MonAggList.append( H0StatsList )
    DataDict = dict()
    iCnt = 0
    for tHdr in StatsHdsList:
        DataDict[tHdr] = np.array( [ x[iCnt] for x in MonAggList ], dtype=np.float32 )
        iCnt += 1
    # end of for
    SummQStatsDF = pd.DataFrame( index=MonthLister, data=DataDict )
    # now write our worksheets
    OutFilePath = os.path.normpath( os.path.join( IN_DIR, "%s_H0CProj1_Precip.xlsx" % OUT_ROOT ) )
    if gCnt == 0:
        with pd.ExcelWriter( OutFilePath, engine="openpyxl" ) as writer:
            SummQStatsDF.to_excel( writer, sheet_name="Summary_%d" % gG, na_rep=str(np.nan), 
                                   columns=StatsHdsList )
            QuantsDF.to_excel( writer, sheet_name="Daily_Stats_%d" % gG, na_rep=str(np.nan), 
                               columns=StatsHdsList )
            DailyQStatsDF.to_excel( writer, sheet_name="DayYear_Stats_%d" % gG, na_rep=str(np.nan), 
                                    columns=StatsHdsList )
            PTMonthly.to_excel( writer, sheet_name="Monthly_Stats_%d" % gG, na_rep=str(np.nan) )
    else:
        with pd.ExcelWriter( OutFilePath, engine="openpyxl", mode='a' ) as writer:
            SummQStatsDF.to_excel( writer, sheet_name="Summary_%d" % gG, na_rep=str(np.nan), 
                                   columns=StatsHdsList )
            QuantsDF.to_excel( writer, sheet_name="Daily_Stats_%d" % gG, na_rep=str(np.nan), 
                               columns=StatsHdsList )
            DailyQStatsDF.to_excel( writer, sheet_name="DayYear_Stats_%d" % gG, na_rep=str(np.nan), 
                                    columns=StatsHdsList )
            PTMonthly.to_excel( writer, sheet_name="Monthly_Stats_%d" % gG, na_rep=str(np.nan) )
    # increment our counter
    gCnt += 1

### H0 CProj Period 2

Expect all of the H0 projection cases to be different.

In [15]:
gCnt = 0
for gG in WGI.LOCA_KEYS:
    InFileName = "%s%d.%s" % ( H0P2_ROOT, gG, PICKLE_EXT )
    InFile = os.path.normpath( os.path.join( IN_DIR, InFileName ) )
    H0DF = pd.read_pickle( InFile, compression='zip' )
    # collect the overall statistics
    H0StatsList = list()
    npFlat = H0DF.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    H0StatsList.extend( QArray.tolist() )
    H0StatsList.append( npFlat.mean() )
    H0StatsList.append( npFlat.var() )
    H0StatsList.append( sstats.skew( npFlat ) )
    H0StatsList.append( sstats.kurtosis( npFlat ) )
    # do probabilistic time history
    DataDict = dict()
    for iI in range(NumQs):
        cQ = OurQs[iI]
        Hdr = "%d_ptile" % round(cQ * 100.0)
        DataDict[Hdr] = np.array( H0DF.quantile(q=cQ, axis=1), dtype=np.float32 )
    # end of for
    QuantsDF = pd.DataFrame( index=H0DF.index, data=DataDict )
    QuantsDF["Average"] = H0DF.mean(axis=1)
    QuantsDF["Variance"] = H0DF.var(axis=1)
    QuantsDF["Skew"] = H0DF.skew(axis=1)
    QuantsDF["Kurt"] = H0DF.kurtosis(axis=1)
    # now do day of the year distributions
    H0DF["IDayYear"] = H0DF.index.dayofyear
    DYList = list()
    for dD in DayYrList:
        LocalList = list()
        cDyOYrDF = H0DF[H0DF["IDayYear"] == dD].copy()
        cDyOYrDF.drop(columns="IDayYear", inplace=True )
        npFlat = cDyOYrDF.to_numpy().flatten(order='F')
        QArray = np.quantile( npFlat, OurQs )
        LocalList.extend( QArray.tolist() )
        LocalList.append( npFlat.mean() )
        LocalList.append( npFlat.var() )
        LocalList.append( sstats.skew( npFlat ) )
        LocalList.append( sstats.kurtosis( npFlat ) )
        DYList.append( LocalList )
    # end of for
    DataDict = dict()
    iCnt = 0
    for tHdr in StatsHdsList:
        DataDict[tHdr] = np.array( [ x[iCnt] for x in DYList ], dtype=np.float32 )
        iCnt += 1
    # end of for
    DailyQStatsDF = pd.DataFrame( index=DayYrIndex, data=DataDict )
    H0DF.drop( columns="IDayYear", inplace=True )
    # make a copy of H0DF for further manipulation
    H0DF["Month"] = H0DF.index.month
    PTH0DF = H0DF.copy()
    PTH0DF["Year"] = PTH0DF.index.year
    PTH0DF["Average"] = PTH0DF[RealColumns].mean(axis=1)
    PTH0DF["Max"] = PTH0DF[RealColumns].max(axis=1)
    PTH0DF["Min"] = PTH0DF[RealColumns].min(axis=1)
    PTH0DF.drop(columns=RealColumns, inplace=True)
    H0grouped = PTH0DF.groupby( ['Year', 'Month'], as_index=True )
    H0TSOutMeanDF = H0grouped[["Average"]].aggregate( [np.mean] )
    H0TSOutMaxDF = H0grouped[["Max"]].aggregate( [np.max] )
    H0TSOutMinDF = H0grouped[["Min"]].aggregate( [np.min] )
    PTMonthly = H0TSOutMaxDF.merge( H0TSOutMeanDF, how='inner', left_index=True, right_index=True )
    PTMonthly = PTMonthly.merge( H0TSOutMinDF, how='inner', left_index=True, right_index=True )
    # split into months
    H0Jan = H0DF[H0DF["Month"] == 1].copy()
    H0Jan.drop(columns="Month", inplace=True)
    H0Feb = H0DF[H0DF["Month"] == 2].copy()
    H0Feb.drop(columns="Month", inplace=True)
    H0Mar = H0DF[H0DF["Month"] == 3].copy()
    H0Mar.drop(columns="Month", inplace=True)
    H0Apr = H0DF[H0DF["Month"] == 4].copy()
    H0Apr.drop(columns="Month", inplace=True)
    H0May = H0DF[H0DF["Month"] == 5].copy()
    H0May.drop(columns="Month", inplace=True)
    H0Jun = H0DF[H0DF["Month"] == 6].copy()
    H0Jun.drop(columns="Month", inplace=True)
    H0Jul = H0DF[H0DF["Month"] == 7].copy()
    H0Jul.drop(columns="Month", inplace=True)
    H0Aug = H0DF[H0DF["Month"] == 8].copy()
    H0Aug.drop(columns="Month", inplace=True)
    H0Sep = H0DF[H0DF["Month"] == 9].copy()
    H0Sep.drop(columns="Month", inplace=True)
    H0Oct = H0DF[H0DF["Month"] == 10].copy()
    H0Oct.drop(columns="Month", inplace=True)
    H0Nov = H0DF[H0DF["Month"] == 11].copy()
    H0Nov.drop(columns="Month", inplace=True)
    H0Dec = H0DF[H0DF["Month"] == 12].copy()
    H0Dec.drop(columns="Month", inplace=True)
    MonAggList = list()
    # Jan
    LocalList = list()
    npFlat = H0Jan.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Feb
    LocalList = list()
    npFlat = H0Feb.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Mar
    LocalList = list()
    npFlat = H0Mar.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Apr
    LocalList = list()
    npFlat = H0Apr.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # May
    LocalList = list()
    npFlat = H0May.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # June
    LocalList = list()
    npFlat = H0Jun.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Jul
    LocalList = list()
    npFlat = H0Jul.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Aug
    LocalList = list()
    npFlat = H0Aug.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Sep
    LocalList = list()
    npFlat = H0Sep.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Oct
    LocalList = list()
    npFlat = H0Oct.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Nov
    LocalList = list()
    npFlat = H0Nov.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Dec
    LocalList = list()
    npFlat = H0Dec.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    MonAggList.append( H0StatsList )
    DataDict = dict()
    iCnt = 0
    for tHdr in StatsHdsList:
        DataDict[tHdr] = np.array( [ x[iCnt] for x in MonAggList ], dtype=np.float32 )
        iCnt += 1
    # end of for
    SummQStatsDF = pd.DataFrame( index=MonthLister, data=DataDict )
    # now write our worksheets
    OutFilePath = os.path.normpath( os.path.join( IN_DIR, "%s_H0CProj2_Precip.xlsx" % OUT_ROOT ) )
    if gCnt == 0:
        with pd.ExcelWriter( OutFilePath, engine="openpyxl" ) as writer:
            SummQStatsDF.to_excel( writer, sheet_name="Summary_%d" % gG, na_rep=str(np.nan), 
                                   columns=StatsHdsList )
            QuantsDF.to_excel( writer, sheet_name="Daily_Stats_%d" % gG, na_rep=str(np.nan), 
                               columns=StatsHdsList )
            DailyQStatsDF.to_excel( writer, sheet_name="DayYear_Stats_%d" % gG, na_rep=str(np.nan), 
                                    columns=StatsHdsList )
            PTMonthly.to_excel( writer, sheet_name="Monthly_Stats_%d" % gG, na_rep=str(np.nan) )
    else:
        with pd.ExcelWriter( OutFilePath, engine="openpyxl", mode='a' ) as writer:
            SummQStatsDF.to_excel( writer, sheet_name="Summary_%d" % gG, na_rep=str(np.nan), 
                                   columns=StatsHdsList )
            QuantsDF.to_excel( writer, sheet_name="Daily_Stats_%d" % gG, na_rep=str(np.nan), 
                               columns=StatsHdsList )
            DailyQStatsDF.to_excel( writer, sheet_name="DayYear_Stats_%d" % gG, na_rep=str(np.nan), 
                                    columns=StatsHdsList )
            PTMonthly.to_excel( writer, sheet_name="Monthly_Stats_%d" % gG, na_rep=str(np.nan) )
    # increment our counter
    gCnt += 1

### H0 CProj Period 3

Expect all of the H0 projection cases to be different.

In [16]:
gCnt = 0
for gG in WGI.LOCA_KEYS:
    InFileName = "%s%d.%s" % ( H0P3_ROOT, gG, PICKLE_EXT )
    InFile = os.path.normpath( os.path.join( IN_DIR, InFileName ) )
    H0DF = pd.read_pickle( InFile, compression='zip' )
    # collect the overall statistics
    H0StatsList = list()
    npFlat = H0DF.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    H0StatsList.extend( QArray.tolist() )
    H0StatsList.append( npFlat.mean() )
    H0StatsList.append( npFlat.var() )
    H0StatsList.append( sstats.skew( npFlat ) )
    H0StatsList.append( sstats.kurtosis( npFlat ) )
    # do probabilistic time history
    DataDict = dict()
    for iI in range(NumQs):
        cQ = OurQs[iI]
        Hdr = "%d_ptile" % round(cQ * 100.0)
        DataDict[Hdr] = np.array( H0DF.quantile(q=cQ, axis=1), dtype=np.float32 )
    # end of for
    QuantsDF = pd.DataFrame( index=H0DF.index, data=DataDict )
    QuantsDF["Average"] = H0DF.mean(axis=1)
    QuantsDF["Variance"] = H0DF.var(axis=1)
    QuantsDF["Skew"] = H0DF.skew(axis=1)
    QuantsDF["Kurt"] = H0DF.kurtosis(axis=1)
    # now do day of the year distributions
    H0DF["IDayYear"] = H0DF.index.dayofyear
    DYList = list()
    for dD in DayYrList:
        LocalList = list()
        cDyOYrDF = H0DF[H0DF["IDayYear"] == dD].copy()
        cDyOYrDF.drop(columns="IDayYear", inplace=True )
        npFlat = cDyOYrDF.to_numpy().flatten(order='F')
        QArray = np.quantile( npFlat, OurQs )
        LocalList.extend( QArray.tolist() )
        LocalList.append( npFlat.mean() )
        LocalList.append( npFlat.var() )
        LocalList.append( sstats.skew( npFlat ) )
        LocalList.append( sstats.kurtosis( npFlat ) )
        DYList.append( LocalList )
    # end of for
    DataDict = dict()
    iCnt = 0
    for tHdr in StatsHdsList:
        DataDict[tHdr] = np.array( [ x[iCnt] for x in DYList ], dtype=np.float32 )
        iCnt += 1
    # end of for
    DailyQStatsDF = pd.DataFrame( index=DayYrIndex, data=DataDict )
    H0DF.drop( columns="IDayYear", inplace=True )
    # make a copy of H0DF for further manipulation
    H0DF["Month"] = H0DF.index.month
    PTH0DF = H0DF.copy()
    PTH0DF["Year"] = PTH0DF.index.year
    PTH0DF["Average"] = PTH0DF[RealColumns].mean(axis=1)
    PTH0DF["Max"] = PTH0DF[RealColumns].max(axis=1)
    PTH0DF["Min"] = PTH0DF[RealColumns].min(axis=1)
    PTH0DF.drop(columns=RealColumns, inplace=True)
    H0grouped = PTH0DF.groupby( ['Year', 'Month'], as_index=True )
    H0TSOutMeanDF = H0grouped[["Average"]].aggregate( [np.mean] )
    H0TSOutMaxDF = H0grouped[["Max"]].aggregate( [np.max] )
    H0TSOutMinDF = H0grouped[["Min"]].aggregate( [np.min] )
    PTMonthly = H0TSOutMaxDF.merge( H0TSOutMeanDF, how='inner', left_index=True, right_index=True )
    PTMonthly = PTMonthly.merge( H0TSOutMinDF, how='inner', left_index=True, right_index=True )
    # split into months
    H0Jan = H0DF[H0DF["Month"] == 1].copy()
    H0Jan.drop(columns="Month", inplace=True)
    H0Feb = H0DF[H0DF["Month"] == 2].copy()
    H0Feb.drop(columns="Month", inplace=True)
    H0Mar = H0DF[H0DF["Month"] == 3].copy()
    H0Mar.drop(columns="Month", inplace=True)
    H0Apr = H0DF[H0DF["Month"] == 4].copy()
    H0Apr.drop(columns="Month", inplace=True)
    H0May = H0DF[H0DF["Month"] == 5].copy()
    H0May.drop(columns="Month", inplace=True)
    H0Jun = H0DF[H0DF["Month"] == 6].copy()
    H0Jun.drop(columns="Month", inplace=True)
    H0Jul = H0DF[H0DF["Month"] == 7].copy()
    H0Jul.drop(columns="Month", inplace=True)
    H0Aug = H0DF[H0DF["Month"] == 8].copy()
    H0Aug.drop(columns="Month", inplace=True)
    H0Sep = H0DF[H0DF["Month"] == 9].copy()
    H0Sep.drop(columns="Month", inplace=True)
    H0Oct = H0DF[H0DF["Month"] == 10].copy()
    H0Oct.drop(columns="Month", inplace=True)
    H0Nov = H0DF[H0DF["Month"] == 11].copy()
    H0Nov.drop(columns="Month", inplace=True)
    H0Dec = H0DF[H0DF["Month"] == 12].copy()
    H0Dec.drop(columns="Month", inplace=True)
    MonAggList = list()
    # Jan
    LocalList = list()
    npFlat = H0Jan.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Feb
    LocalList = list()
    npFlat = H0Feb.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Mar
    LocalList = list()
    npFlat = H0Mar.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Apr
    LocalList = list()
    npFlat = H0Apr.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # May
    LocalList = list()
    npFlat = H0May.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # June
    LocalList = list()
    npFlat = H0Jun.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Jul
    LocalList = list()
    npFlat = H0Jul.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Aug
    LocalList = list()
    npFlat = H0Aug.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Sep
    LocalList = list()
    npFlat = H0Sep.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Oct
    LocalList = list()
    npFlat = H0Oct.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Nov
    LocalList = list()
    npFlat = H0Nov.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Dec
    LocalList = list()
    npFlat = H0Dec.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    MonAggList.append( H0StatsList )
    DataDict = dict()
    iCnt = 0
    for tHdr in StatsHdsList:
        DataDict[tHdr] = np.array( [ x[iCnt] for x in MonAggList ], dtype=np.float32 )
        iCnt += 1
    # end of for
    SummQStatsDF = pd.DataFrame( index=MonthLister, data=DataDict )
    # now write our worksheets
    OutFilePath = os.path.normpath( os.path.join( IN_DIR, "%s_H0CProj3_Precip.xlsx" % OUT_ROOT ) )
    if gCnt == 0:
        with pd.ExcelWriter( OutFilePath, engine="openpyxl" ) as writer:
            SummQStatsDF.to_excel( writer, sheet_name="Summary_%d" % gG, na_rep=str(np.nan), 
                                   columns=StatsHdsList )
            QuantsDF.to_excel( writer, sheet_name="Daily_Stats_%d" % gG, na_rep=str(np.nan), 
                               columns=StatsHdsList )
            DailyQStatsDF.to_excel( writer, sheet_name="DayYear_Stats_%d" % gG, na_rep=str(np.nan), 
                                    columns=StatsHdsList )
            PTMonthly.to_excel( writer, sheet_name="Monthly_Stats_%d" % gG, na_rep=str(np.nan) )
    else:
        with pd.ExcelWriter( OutFilePath, engine="openpyxl", mode='a' ) as writer:
            SummQStatsDF.to_excel( writer, sheet_name="Summary_%d" % gG, na_rep=str(np.nan), 
                                   columns=StatsHdsList )
            QuantsDF.to_excel( writer, sheet_name="Daily_Stats_%d" % gG, na_rep=str(np.nan), 
                               columns=StatsHdsList )
            DailyQStatsDF.to_excel( writer, sheet_name="DayYear_Stats_%d" % gG, na_rep=str(np.nan), 
                                    columns=StatsHdsList )
            PTMonthly.to_excel( writer, sheet_name="Monthly_Stats_%d" % gG, na_rep=str(np.nan) )
    # increment our counter
    gCnt += 1